In [5]:
import pandas as pd
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier  # MUDANÇA AQUI
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np

# --- Passo 1: Carregar os Dados Limpos ---
caminho_dados_limpos = os.path.join('..', 'data', 'dados_limpos.csv')

try:
    df = pd.read_csv(caminho_dados_limpos)
    print("Arquivo de dados limpos carregado com sucesso!")
except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado em '{caminho_dados_limpos}'.")

# --- Passo 2: Preparação dos Dados para o Modelo ---
df_modelo = df[df['risco_evasao_declarado'].isin(['Sim', 'Não'])].copy()

features = [
    'curso', 'periodo', 'idade', 'genero', 'cor_raca',
    'renda_familiar', 'trabalha'
]

target = 'risco_evasao_declarado'

X = df_modelo[features]
y = df_modelo[target]

X_encoded = pd.get_dummies(X, drop_first=True)
model_columns = X_encoded.columns
joblib.dump(model_columns, os.path.join('..', 'dashboard', 'colunas_modelo.joblib'))

# --- APLICAR SMOTE ---
print("\nAplicando SMOTE para balancear os dados...")
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_encoded, y)

print(f"Distribuição original: {np.bincount([1 if val == 'Sim' else 0 for val in y])}")
print(f"Distribuição após SMOTE: {np.bincount([1 if val == 'Sim' else 0 for val in y_balanced])}")

# --- Dividir os Dados ---
X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced
)

print(f"\nTamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")

# --- VALIDAÇÃO CRUZADA COM GRADIENT BOOSTING ---
print("\nRealizando validação cruzada...")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model_cv = GradientBoostingClassifier(  # MUDANÇA AQUI
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42
)
cv_scores = cross_val_score(model_cv, X_balanced, y_balanced, cv=cv, scoring='f1_macro')
print(f"F1-Score médio na validação cruzada: {cv_scores.mean():.2f} (+/- {cv_scores.std() * 2:.2f})")

# --- TREINAR GRADIENT BOOSTING ---
print("\nIniciando o treinamento do modelo GradientBoostingClassifier...")  # MUDANÇA AQUI

model = GradientBoostingClassifier(  # MUDANÇA AQUI
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42
)

model.fit(X_train, y_train)
print("Treinamento concluído!")

# --- AVALIAR O MODELO ---
print("\nAvaliando o modelo com os dados de teste...")
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"\nAcurácia do modelo: {accuracy:.2f}")

print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# --- SALVAR O MODELO ---
caminho_modelo = os.path.join('..', 'dashboard', 'modelo_evasao.joblib')
joblib.dump(model, caminho_modelo)
print(f"\nModelo salvo com sucesso em: '{caminho_modelo}'")

Arquivo de dados limpos carregado com sucesso!

Aplicando SMOTE para balancear os dados...
Distribuição original: [38 19]
Distribuição após SMOTE: [38 38]

Tamanho do conjunto de treino: 60 amostras
Tamanho do conjunto de teste: 16 amostras

Realizando validação cruzada...
F1-Score médio na validação cruzada: 0.62 (+/- 0.18)

Iniciando o treinamento do modelo GradientBoostingClassifier...
Treinamento concluído!

Avaliando o modelo com os dados de teste...

Acurácia do modelo: 0.62

Matriz de Confusão:
[[5 3]
 [3 5]]

Relatório de Classificação:
              precision    recall  f1-score   support

         Não       0.62      0.62      0.62         8
         Sim       0.62      0.62      0.62         8

    accuracy                           0.62        16
   macro avg       0.62      0.62      0.62        16
weighted avg       0.62      0.62      0.62        16


Modelo salvo com sucesso em: '..\dashboard\modelo_evasao.joblib'
